# GRU(Gated Recurrent Units) 모델
- https://arxiv.org/pdf/1406.1078
- LSTM이 RNN의 한계점인 기억력 소실문제를 해결하여 긴 sequence의 데이터에서도 좋은 성능을 내는 모델이다. 그러나 복잡한 구조로 parameter가 많아지게 되었고 연산량이 많은 문제점이 있다.
    - parameter가 많아지면서 데이터양이 부족할 경우 과대적합이 발생하고 연산량이 많아 학습에 많은 시간이 걸리게 된다.
- LSTM의 이런 문제를 개선하기 위한 변형 모델이 GRU이다.

## LSTM과 차이
1. LSTM은 forget gate, input gate, output gete 세개의 Gate연산을 함. GRU는 **reset get와 update gate** 로 흐름을 제어한다.
2. LSTM은 이전 처리결과로 Cell State, Hidden State 두개가 있었는데 이것을 하나로 합쳐 **Hidden State**로 출력한다.

## GRU 성능
- GRU는 적은 파라미터 수와 연산비용이 적게 드는 것에 비해 LSTM과 비슷한 성능을 내는 것으로 알려졌다.

## GRU Cell 구조

![gru_cell](figures/rnn/23_gru_cell.png)    
[이미지 Source](https://www.oreilly.com/library/view/advanced-deep-learning/9781789956177/8ad9dc41-3237-483e-8f6b-7e5f653dc693.xhtml)

- **Reset Gate**
    - 이전 timestep의 hidden state값을 현재 timestep의 hidden state 계산시 얼마나 반영할 지를 결정하는 gate
    - $r_{t} = \sigma(h_{t-1}\cdot U_{r} + X_{t}\cdot W_{r})$
        - $U_{r},\, W_{r}$ 는 파라미터
        - $\sigma$: sigmoid(logisic) 함수
- **Update Gate**
    - 현재 timestep의 hidden state($h_t$)를 계산할 때 이전 time step까지 정보($h_{t-1}$)와 현재 time step의 정보($X_t$)를 각각 얼마나 반영할지 비율을 정의한다.
    - $z_{t} = \sigma(h_{t-1}\cdot U_{z} + X_{t}\cdot W_{z})$
        - $U_{z},\, W_{z}$ 는 파라미터
        - $\sigma$: sigmoid(logisic) 함수
    - $h_t$를 계산할 때 $z_{t}$ 는 이전 정보인 $h_{t-1}$을 얼마나 반영할지 $1-z_{t}$는 현재 정보를 얼마나 반영할 지를 정한다.
- **Cell의 출력값인 $h_t$ 계산**
    - $z_{t}\times h_{t-1} + tanh(h_{t-1} * r_{t}+X_{t}\cdot W)\times(1-z_{t})$
    - 이전 정보에는 $z_t$를 곱해 얼마나 $h_t$ 에 더할지 연산
    - 현재 정보($X_t$)에는 이전 정보를 일부를 반영한다. 이전 정보를 얼마나 반영할지를 reset gate 결과를 곱해 결정한다. 활성화 함수 tanh를 이용해 비선형성을 추가 한 결과에 $1-z_t$를 곱한다.

## Pytorch GRU
- `nn.GRU` 클래스 이용
    - https://pytorch.org/docs/stable/generated/torch.nn.GRU.html
- **입력**
    - **input**: (seq_length, batch, hidden_size) shape의 tensor. (batch_first=False), batch_first=True이면 `seq_length`와 `batch` 위치가 바뀐다.
    - **hidden**: (D * num_layers, batch, hidden_size) shape의 Tensor. D(양방향:2, 단방향:1), hidden은 생략하면 0이 입력됨.
- **출력** - output과 hidden state가 반환된다.
    - **output**
        - 모든 sequence의 처리결과들을 모아서 제공.
        - shape: (seq_length, batch, D * hidden_size) : D(양방향:2, 단방향:1), batch_first=True이면 `seq_length`와 `batch` 위치가 바뀐다.
    - **hidden**
        - 마지막 time step 처리결과
        - shape: (D * num_layers, batch, hidden) : D(양방향:2, 단방향:1)

In [ ]:
import torch
import torch.nn as nn

input_data = torch.randn((20, 2, 10))   # [seq_len,  batch_size,  input_size] (float)

In [ ]:
gru1 = nn.GRU(
    input_size=10, 
    hidden_size=30    #num_layers=1, bd=False, batch_first=False
)
o1, h1 = gru1(input_data)  # input, hidden(생략=>0, 첫번째 timestep의 hidden)
print(o1.shape)  # 모든 timestep의 처리결과를 묶어서 반환. [seq_len, batch, hidden_size*D]
print(h1.shape)  # 마지막 timestep의 처리결과. 방향별, layer별 hidden state값을 묶어서 제공.
#  [D * num_layers,   batch, hidden_size]

In [ ]:

gru2 = nn.GRU(
    input_size=10, 
    hidden_size=30, 
)
o2, h2 = gru2(input_data, h1)  # input, hidden (앞 GRU의 hidden을 다음 GRU에 입력)
print(o2.shape)  # 모든 timestep의 처리결과를 묶어서 반환. [seq_len, batch, hidden_size*D]
print(h2.shape)  # 마지막 timestep의 처리결과. 방향별, layer별 hidden state값을 묶어서 제공.
#  [D * num_layers,   batch, hidden_size]

In [ ]:
gru3 = nn.GRU(
    input_size=10, 
    hidden_size=30 ,
    num_layers=3,
    bidirectional=True
)
o3, h3 = gru3(input_data)  # input, hidden(생략=>0)
print(o3.shape)  # 모든 timestep의 처리결과를 묶어서 반환. [seq_len, batch, hidden_size*2]
print(h3.shape)  # 마지막 timestep의 처리결과. 방향별, layer별 hidden state값을 묶어서 제공.
#  [1* 3,   batch, hidden_size]

# Encoder-Decoder 구조
- 두개의 네트워크를 연결한 구조
- Encoder network는 입력을 이해하고 Decoder network는 (Encoder의 이해를 바탕으로) 출력을 생성한다.

## Seq2Seq
- Encoder-Decoder 구조를 RNN 계열에 적용한 모델.
- Encoder는 입력 Sequence의 전체 의미(특징)을 표현하는 **context vector**를 출력한다.
    - **Context Vector는**
        - 번역의 경우 번역할 대상 문장에서 **번역 결과를 만들때 필요한 feature들**을 가지고 있다.
        - Chatbot의 경우 입력된 질문에서 **답변을 만들때 필요한 feature들**을 가지고 있다.
- Decoder는 Encoder가 출력한 Context Vector를 입력받아 결과 sequence를 생성한다.
    - **결과 sequence는**
        - **번역**의 경우 번역 문장을 생성한다.
        - **chatbot**의 경우 질문에 대한 답변을 생성한다.

![seq2seq](figures/seq2seq.png)

# Seq2Seq 를 이용한 Chatbot 모델 구현
- Encoder를 이용해 질문의 특성을 추출하고 Decoder를 이용해 답변을 생성한다.

# Chatbot Dataset

- https://github.com/songys/Chatbot_data
- columns
    - Q: 질문
    - A: 답
    - label: 일상다반사 0, 이별(부정) 1, 사랑(긍정) 2
- **Download**

![dataset](figures/chatbot.png)

# Chatbot Dataset Loading 및 확인

## 데이터 확인

In [1]:
import pandas as pd

df = pd.read_csv("datasets/ChatbotData.csv")
df.shape

(11823, 3)

In [2]:
df.head()

,Q,A,label
0,12시 땡!,하루가 또 가네요.,0
1,1지망 학교 떨어졌어,위로해 드립니다.,0
2,3박4일 놀러가고 싶다,여행은 언제나 좋죠.,0
3,3박4일 정도 놀러가고 싶다,여행은 언제나 좋죠.,0
4,PPL 심하네,눈살이 찌푸려지죠.,0


In [3]:
df.drop(columns="label", inplace=True)
df.shape

(11823, 2)

In [4]:
df.head()

,Q,A
0,12시 땡!,하루가 또 가네요.
1,1지망 학교 떨어졌어,위로해 드립니다.
2,3박4일 놀러가고 싶다,여행은 언제나 좋죠.
3,3박4일 정도 놀러가고 싶다,여행은 언제나 좋죠.
4,PPL 심하네,눈살이 찌푸려지죠.


In [5]:
df.tail()

,Q,A
11818,훔쳐보는 것도 눈치 보임.,티가 나니까 눈치가 보이는 거죠!
11819,훔쳐보는 것도 눈치 보임.,훔쳐보는 거 티나나봐요.
11820,흑기사 해주는 짝남.,설렜겠어요.
11821,힘든 연애 좋은 연애라는게 무슨 차이일까?,잘 헤어질 수 있는 사이 여부인 거 같아요.
11822,힘들어서 결혼할까봐,도피성 결혼은 하지 않길 바라요.


In [6]:
df.isna().sum()

Q    0
A    0
dtype: int64

# Dataset, DataLoader 정의

## Tokenization

### Subword방식

In [7]:
## 질문 문장 + 답변 문장 => tokenizer 객체를 생성.
question_texts = list(df['Q'])  # list(Series)
answer_texts = list(df['A'])
all_texts = question_texts + answer_texts
len(all_texts), len(question_texts), len(answer_texts)

(23646, 11823, 11823)

In [9]:
all_texts[:5]

['12시 땡!', '1지망 학교 떨어졌어', '3박4일 놀러가고 싶다', '3박4일 정도 놀러가고 싶다', 'PPL 심하네']

In [10]:
import os
from tokenizers import Tokenizer
from tokenizers.models import BPE
from tokenizers.pre_tokenizers import Whitespace
from tokenizers.trainers import BpeTrainer

vocab_size = 30000
min_frequency = 2

tokenizer = Tokenizer(
    BPE(unk_token='[UNK]')  # Unknow Token(Out Of Vocabulary) - 어휘사전에 없는 단어.
)
tokenizer.pre_tokenizer = Whitespace()
trainer = BpeTrainer(  # 학습 방식을 설정.
    vocab_size=vocab_size,
    min_frequency=min_frequency, 
    special_tokens=["[PAD]", "[UNK]", "[SOS]", "[EOS]"], 
    continuing_subword_prefix="#", 
    # 단어 중간에 사용된 subword일 경우 앞에 붙일 표시특수문자.
    #   돌부처 => 돌, #부처    부처님 => 부처, #님
)
# 학습 
##  Dataset이 메모리에 있는 경우. ["문서", "문서", "문서",  ......]
tokenizer.train_from_iterator(all_texts, trainer=trainer)

In [11]:
print("총어휘수:", tokenizer.get_vocab_size())
encode = tokenizer.encode("내일은 즐거운 주말입니다.")
print(encode.tokens)
print(encode.ids)
print(tokenizer.id_to_token(3842))
print(tokenizer.token_to_id('내일은'))
print(tokenizer.token_to_id('입니다'), tokenizer.token_to_id('#입니다'))

총어휘수: 15579
['내일은', '즐거운', '주말', '#입니다', '.']
[3842, 3272, 2806, 2467, 9]
내일은
3842
8871 2467


In [12]:
tokenizer.token_to_id("[PAD]")

0

### Tokenizer 저장

In [13]:
os.makedirs('models/tokenizers', exist_ok=True)
tk_path = 'models/tokenizers/chatbot_data_bpe.json'
tokenizer.save(tk_path)

# 로딩
## load_tokenizer = Tokenizer.from_file(tk_path)

## Dataset, DataLoader 정의


### Dataset 정의 및 생성
- 모든 문장의 토큰 수는 동일하게 맞춰준다.
    - DataLoader는 batch 를 구성할 때 batch에 포함되는 데이터들의 shape이 같아야 한다. 그래야 하나의 batch로 묶을 수 있다.
    - 문장의 최대 길이를 정해주고 **최대 길이보다 짧은 문장은 `<PAD>` 토큰을 추가**하고 **최대길이보다 긴 문장은 최대 길이에 맞춰 짤라준다.**

In [14]:
import random
import os
import numpy as np
import time

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader, random_split, SubsetRandomSampler
from torch import optim

device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cpu'

In [16]:
# train_set[0]  => (x[0]-질문,  y[0]-답변)
# x, y -> 토큰_id 리스트.  [3842, 3271, 2806, 2467, 9, PAD, PAD, ...]
class ChatbotDataset(Dataset):
    """
    ChatbotDataset
    Attribute
        question_texts: list[str] - 질문 texts 목록. 리스트에 질문들을 담아서 받는다. ["질문1", "질문2", ...]
        answer_texts: list[str] - 답 texts 목록. 리스트에 답변들을 담아서 받는다.     ["답1", "답2", ...]
        max_length: 개별 댓글의 token 개수. 모든 댓글의 토큰수를 max_length에 맞춘다.
        tokenizer: Tokenizer
        vocab_size: int 총단어수
        PAD_TOKEN: int Padding 토큰 id
    """
    def __init__(self, question_texts, answer_texts, max_length, tokenizer):
        """
        Parameter
            question_texts: list[str] - 질문 texts 목록. 리스트에 질문들을 담아서 받는다. ["질문1", "질문2", ...]
            answer_texts: list[str] - 답 texts 목록. 리스트에 답변들을 담아서 받는다.     ["답1", "답2", ...]
            max_length: 개별 댓글의 token 개수. 모든 댓글의 토큰수를 max_length에 맞춘다.
            tokenizer: Tokenizer
        """
        self.question_texts = question_texts
        self.answer_texts = answer_texts
        self.max_length = max_length
        self.tokenizer = tokenizer
        
        self.vocab_size = tokenizer.get_vocab_size()
        self.PAD_TOKEN = self.tokenizer.token_to_id("[PAD]")
    
    def __pad_token_sequence(self, token_sequence):
        """
        max_length 길이에 맞춰 token_id 리스트를 구성한다.
        max_length 보다 길면 뒤에를 자르고 max_length 보다 짧으면 [PAD] 토큰을 추가한다.
        
        Parameter
            token_sentence: list[int] - 길이를 맞출 한 문장 token_id 목록
        Return
            list[int] - length가 max_length인 token_id 목록
        """
        seq_len = len(token_sequence)
        token_ids = None
        if seq_len >= self.max_length: # 크면 자르기
            token_ids = token_sequence[:self.max_length]
        else: # 적으면 PAD 추가.
            token_ids = token_sequence + ([self.PAD_TOKEN] * (self.max_length - seq_len))
        return token_ids
        
    def __process_sequence(self, text):
        """
        한문장을 받아서 padding이 추가된 token_id 리스트로 변환 후 반환
        Parameter
            text: str - token_id 리스트로 변환할 한 문장
        Return
            list[int] - 입력받은 문장에 대한 token_id 리스트
        """
        # 문장 -> encoding
        encode = self.tokenizer.encode(text)
        # encode.ids의 크기를 max_length에 맞춘다. 
        token_ids = self.__pad_token_sequence(encode.ids) # [ 30, 20, 10] => [30, 20, 10, 0, 0, ..]
        return token_ids
    
    def __len__(self):
        return len(self.question_texts)
    
    def __getitem__(self, index):
        # index의 question, anwser token_id_리스트를 묶어서 반환.
        question = self.question_texts[index]  # string
        answer = self.answer_texts[index]

        # encoding+padding 처리된 token_id_list: list 생성.
        # "학교는 어디있나요?"  ==> [2000, 323, 2131, 3908, ...]
        question_token_ids = self.__process_sequence(question) 
        answer_token_ids = self.__process_sequence(answer)

        return torch.LongTensor(question_token_ids), torch.LongTensor(answer_token_ids)

In [17]:
from tokenizers import Tokenizer

MAX_LENGTH = 25
tokenizer = Tokenizer.from_file('models/tokenizers/chatbot_data_bpe.json')

dataset = ChatbotDataset(question_texts, answer_texts, MAX_LENGTH, tokenizer)
print(dataset.vocab_size)

15579


In [22]:
len(dataset)

11823

In [21]:
a, b = dataset[110]
print(a.shape, a)
print(b.shape, b)

torch.Size([25]) tensor([13130,  4368,   810,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0])
torch.Size([25]) tensor([5679, 4783,   77, 2447,    9,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0])


### Trainset / Testset 나누기
train : test = 8 : 2

In [23]:
len(dataset)

11823

In [24]:
train_size = int(len(dataset) * 0.8)
test_size = len(dataset) - train_size
print(train_size, test_size)

9458 2365


In [25]:
train_set, test_set = random_split(dataset, [train_size, test_size])
len(train_set), len(test_set)

(9458, 2365)

### DataLoader 생성

In [26]:
BATCH_SIZE = 64
train_loader = DataLoader(train_set, batch_size=BATCH_SIZE, shuffle=True, drop_last=True)
test_loader = DataLoader(test_set, batch_size=BATCH_SIZE)
len(train_loader), len(test_loader)

(147, 37)

# 모델 정의

## Seq2Seq 모델 정의
- Seq2Seq 모델은 Encoder와 Decoder의 입력 Sequence의 길이와 순서가 자유롭기 때문에 챗봇이나 번역에 이상적인 구조다.
    - 단일 RNN은 각 timestep 마다 입력과 출력이 있기 때문에 입/출력 sequence의 개수가 같아야 한다.
    - 챗봇의 질문/답변이나 번역의 대상/결과 문장의 경우는 사용하는 어절 수가 다른 경우가 많기 때문에 단일 RNN 모델은 좋은 성능을 내기 어렵다.
    - Seq2Seq는 **입력처리(질문,번역대상)처리 RNN과 출력 처리(답변, 번역결과) RNN 이 각각 만들고 그 둘을 연결한 형태로 길이가 다르더라도 상관없다.**

## Encoder
Encoder는 하나의 Vector를 생성하며 그 Vector는 **입력 문장의 의미**를 N 차원 공간 저장하고 있다. 이 Vector를 **Context Vector** 라고 한다.    
![encoder](figures/seq2seq_encoder.png)

In [ ]:
tokenizer.token_to_id("[PAD]")

In [27]:
class Encoder(nn.Module):
    def __init__(self, vocab_size, hidden_size, embedding_dim, 
                   num_layers=1, bidirectional=False, dropout_rate=0.0):
        super().__init__()
        # 어휘수 저장.
        self.vocab_size = vocab_size
        #Embedding Layer 생성
        self.embedding = nn.Embedding(
            num_embeddings=vocab_size,      #어휘사전의 총 단어수
            embedding_dim=embedding_dim, # 단어 임베딩 벡터의 차원수(몇개 숫자로 구성 할지)
            padding_idx=0
        )
        # GRU
        self.gru = nn.GRU(
            input_size=embedding_dim, 
            hidden_size=hidden_size,
            num_layers=num_layers,
            dropout=dropout_rate,     # dropout=(0.0 if num_layers==1 else dropout_rate)
            bidirectional=bidirectional
        )
    
    def forward(self, X):
        embedding = self.embedding(X)
        # (batch, seq, 임베딩차원)
        embedding = embedding.permute(1, 0, 2)
        # gru
        output, hidden = self.gru(embedding)  
        # output: 모든 timestep의 hidden state들. hidden: 마지막 timestep의 hidden state
        return output, hidden

In [28]:
vocab_size = tokenizer.get_vocab_size()
embedding_dims = 100
hidden_size = 256
bidir = True

a, b = next(iter(train_loader))  # a: Q, b: A
e = Encoder(vocab_size=vocab_size, 
            embedding_dim=embedding_dims, 
            hidden_size=hidden_size, 
            bidirectional=bidir)
o1, h1 = e(a)

In [30]:
print(a.shape) # [64, 25] -> [batch_size, seq_length]
print(b.shape)

torch.Size([64, 25])
torch.Size([64, 25])


In [32]:
print(o1.shape) # [25, 64, 512] -> [seq_length, batch_size, hidden_state * 양방향(2)]
print(h1.shape) # [2, 64, 256]  -> [양방향(2) *  num_layers(1),  batch_size,  hidden_size]

torch.Size([25, 64, 512])
torch.Size([2, 64, 256])


## Decoder
- Encoder의 출력(context vector)를 받아서 번역 결과 sequence를 출력한다.
- Decoder는 매 time step의 입력으로 **이전 time step에서 예상한 단어와 hidden state값이** 입력된다.
- Decoder의 처리결과 hidden state를 Estimator(Linear+Softmax)로 입력하여 **입력 단어에 대한 번역 단어가 출력된다.** (이 출력단어가 다음 step의 입력이 된다.)
    - Decoder의 첫 time step 입력은 문장의 시작을 의미하는 <SOS>(start of string) 토큰이고 hidden state는 context vector(encoder 마지막 hidden state) 이다.

![decoder](figures/seq2seq_decoder.png)

In [34]:
class Decoder(nn.Module):
    def __init__(self, vocab_size, hidden_size, embedding_dim, num_layers=1,
                   dropout_rate=0):
        super().__init__()
        self.vocab_size = vocab_size
        # Embedding layer
        self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx=0)
        # Dropout layer
        self.dropout = nn.Dropout(dropout_rate)
        # GRU - 생성 => 스스로 만든 단어를 다음 step의 입력 --> 단방향 RNN
        self.gru = nn.GRU(
            embedding_dim, 
            hidden_size, 
            num_layers=num_layers,
            dropout=dropout_rate     # dropout=(0 if num_layer==1 else dropout_rate)
        )
        ## 입력에 대해서 예측한 다음 입력 단어
        ### gru: output -> (seq_len: 1, batch, hidden_size*단방향)
        self.classifier = nn.Linear(hidden_size, vocab_size)
        
    def forward(self, X, hidden_state):
        """
        이전 생성 단어와 이전 step까지의 hidden_state를 입력 받아 다음 단어를 생성한다.
        X: 이전 처리단어의 token. 첫번째 time step에서는 [SOS] 문장시작 token을 받는다.
        Parameter:
            X: torch.Tensor - long(64bit) or int(32bit) type Tensor, Token list, shape - [batch]. seq_length는 1 (한글자씩 입력된다.)
            hidden_state: torch.Tensor - Float. 첫번째 timestep의 경우 Encoder의 hidden state
                                                        두번째 timestep 부터는 이전 처리 결과.
                                                        shape: [1, batch, hidden_state] seq_len: 1 - 한개 토큰씩 처리.
        Return
            tuple - 분류결과, hidden_state(RNN 마지막 step 처리결과)  
                    분류결과: 생성할 단어(토큰)의 확률
                    hidden_state: 다음 sequence 처리의 hidden state로 입력.
        """
        # X.shape : [batch] - [64]  한개 토큰만 입력
        X = X.unsqueeze(1)  # [64] =>  [64, 1 :seq_len]   # [batch] -> [batch, 1]
        embedding = self.embedding(X).permute(1, 0, 2)
        # embedding:  [64, 1, 100] [batch_size, seq_len, embedding_dim]
        #    embedding 결과를 GRU에 입력하기 위해 batch_size <->seq_len 

        out = self.dropout(embedding)
        
        output, hidden = self.gru(out, hidden_state)
        ## 마지막 timestep의 처리결과를 Linear에 입력
        # output shape: [seq_len: 1, batch, hidden_size]
        last_out = output[-1, : , :]  # [1, 64, 256] =조회결과=> [64, 256]
        last_out = self.classifier(last_out)
        return last_out, hidden  # 다음단어, 현재 처리 feature(gru 반환 hidden)

In [42]:
d = Decoder(vocab_size, hidden_size * 2,  # encoder hidden state 크기  (양방향-hs * 2)
                 embedding_dims)
i = torch.ones(size=(64, ), dtype=torch.long)# 첫번째 토큰
# print(o1[-1].shape, i.shape) 
o1[-1, :, :].unsqueeze(0).shape  # 25 seq에서 마지막 seq 조회. seq_length dummy axis를 추가.
o2, h2 = d(i, o1[-1, :, :].unsqueeze(0))

In [60]:
o2.shape # [batch_size, 어휘시]
o2[1].topk(2)

torch.return_types.topk(
values=tensor([0.3608, 0.2930], grad_fn=<TopkBackward0>),
indices=tensor([11874,   333]))

In [61]:
o2.shape
# o2.argmax(-1)
o2[:2].topk(3)   # Tensor 원소중 가장 큰값 순서대로 1개 의 값과 index를 반환.
# 가장 큰값 top k개를 반환. (값, index)

torch.return_types.topk(
values=tensor([[0.3608, 0.2930, 0.2531],
        [0.3608, 0.2930, 0.2531]], grad_fn=<TopkBackward0>),
indices=tensor([[11874,   333,  3170],
        [11874,   333,  3170]]))

In [50]:
tokenizer.id_to_token(11874)
tokenizer.decode([11874,   333,  3170])

'팀워 둠 친구한테'

In [53]:
a

tensor([[ 1,  2,  3],
        [10,  2,  5]])

In [64]:
a = torch.tensor(
    [
        [1, 2, 3], 
        [10, 2, 5]
    ]) 

a.topk(k=2)  # 가장 큰값  k개 조회. 

torch.return_types.topk(
values=tensor([[ 3,  2],
        [10,  5]]),
indices=tensor([[2, 1],
        [0, 2]]))

In [48]:
h2.shape

torch.Size([1, 64, 512])

## Seq2Seq 모델

- Encoder - Decoder 를 Layer로 가지며 Encoder로 질문의 feature를 추출하고 Decoder로 답변을 생성한다.

### Teacher Forcing
- **Teacher forcing** 기법은, RNN계열 모델이 다음 단어를 예측할 때, 이전 timestep에서 예측된 단어를 입력으로 사용하는 대신 **실제 정답 단어(ground truth) 단어를** 입력으로 사용하는 방법이다.
    - 모델은 이전 시점의 출력 단어를 다음 시점의 입력으로 사용한다. 그러나 모델이 학습할 때 초반에는 정답과 많이 다른 단어가 생성되어 엉뚱한 입력이 들어가 학습이 빠르게 되지 않는 문제가 있다.
- **장점**
    - **수렴 속도 증가**: 정답 단어를 사용하기 때문에 모델이 더 빨리 학습할 수있다.
    - **안정적인 학습**: 초기 학습 단계에서 모델의 예측이 불안정할 때, 잘못된 예측으로 인한 오류가 다음 단계로 전파되는 것을 막아줍니다.
- **단점**
    - **노출 편향(Exposure Bias) 문제:** 실제 예측 시에는 정답을 제공할 수 없으므로 모델은 전단계의 출력값을 기반으로 예측해 나가야 한다. 학습 과정과 추론과정의 이러한 차이 때문에 모델의 성능이 떨어질 수있다.
        - 이런 문제를 해결하기 학습 할 때 **Teacher forcing을 random하게 적용하여 학습시킨다.**
![seq2seq](figures/seq2seq.png)

In [65]:
tokenizer.token_to_id('[SOS]')

2

In [66]:
a, b = next(iter(train_loader))
a.shape, b.shape

(torch.Size([64, 25]), torch.Size([64, 25]))

In [67]:
SOS_TOKEN = tokenizer.token_to_id('[SOS]')

class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder, device):
        super().__init__()
        self.encoder = encoder.to(device)
        self.decoder = decoder.to(device)
        self.device = device
        
    def forward(self, inputs, outputs, teacher_forcing_ratio=0.5):
        """
        Encoder를 이용해 질문 문장의 특성을 추출한다.
           이 특성을 Decoder의 initial hidden_state로, 
           initial input으로 [SOS] 토큰을 입력하여 순차적으로 답변을 생성한다.
        Encoder는 한번에 특성추출을 진행된다.
        Decoder는 time step 별로 순차적으로 진행된다.
        
        Parameter
            inputs: 질문 batch data
            outputs: 답변 batch data (teacher forcing 시 사용.)
            teacher_forcing_ratio=0.5: float - 추론시 Teacher Forcing(정답을 다음 시점의 입력으로 넣는 것) 이 발생할 비율. 
        """
        # inputs : (batch_size, sequence_length) 질문
        # outputs: (batch_size, sequence_length) 답변
        
        batch_size, output_length = outputs.shape
        output_vocab_size = self.decoder.vocab_size
        
        # 리턴할 예측된 outputs를 저장할 임시 변수
        #                      =>Decoder가 예측한 timestep별 단어를 저장.
        # (sequence_length, batch_size, vocab_size)
        predicted_outputs = torch.zeros(output_length, batch_size, output_vocab_size).to(self.device)
        
        ##############################################
        # Encoder  처리
        ###############################################
        # output [seq_length, batch, hidden * 2] 양방향(* 2) ||| hidden (Bidirectional(2) x number of layers(1), batch_size, hidden_size)
        encoder_output, encoder_hidden = self.encoder(inputs)


        ###################################################
        #  Decoder 처리
        ################################################
        ## encoder_output에서 마지막 timestep의 hidden이 
        ####                      decoder에 입력할 초기 hidden_state (context vector 조회)
        decoder_hidden = encoder_output[-1, :, :].unsqueeze(0)  # [seq_len, batch, hidden] 마지막 seq만 조회. 2차원이 되므로 seq_length 축 추가.
        # [25, 64, 512] = [-1, :, :] => [64, 512] =seq_len 축 추가(unsqueeze)=> [1, 64, 512]
       
        # (batch_size) shape의 SOS TOKEN으로 채워진 Decoder 첫번째 timestep의 입력 생성
        decoder_input = torch.full((batch_size,), fill_value=SOS_TOKEN, device=self.device)
        
        # 한개 토큰씩 순회하면서 출력 단어토큰을 생성
        for t in range(0, output_length):
            # decoder_input : 첫번째 입력은 (batch_size) 형태의 [SOS] TOKEN로 채워진 입력, 다음 부터는 생성된 token
            # decoder_output: (batch_size, vocab_size)
            # decoder_hidden: (Bidirectional(1) x number of layers(1), batch_size, hidden_size), context vector와 동일 shape
            decoder_output, decoder_hidden = self.decoder(decoder_input, decoder_hidden)

            # t번째 단어에 디코더의 output 저장
            predicted_outputs[t] = decoder_output
            
            # teacher forcing 적용 여부 확률로 결정 (math.random.random(): 0 ~ 1 실수.)
            teacher_force = random.random() < teacher_forcing_ratio
            
            # top1 단어 토큰 예측
            top1 = decoder_output.argmax(1) 
            
            # teacher forcing 인 경우 ground truth 값을, 그렇지 않은 경우, 예측 값을 다음 input으로 지정
            decoder_input = outputs[:, t] if teacher_force else top1
        
        return predicted_outputs.permute(1, 0, 2) # (batch_size, sequence_length, vocab_size)로 변경

# 학습

## 모델생성

In [68]:
VOCAB_SIZE = dataset.vocab_size

# 인코더 양방향 여부 (True)
ENCODER_BIDIRECTIONAL = True
##  인코더 GRU의 bidirectional=True 인 경우 hidden이 두배로 나온다. 
#   이게 Decoder에 입력되야 하므로 Decoder의 hidden_size는 인코더의 두배가 된다.
ENCODER_HIDDEN_SIZE = 512 # 인코더의 hidden_size
DECODER_HIDDEN_SIZE = ENCODER_HIDDEN_SIZE * 2 if ENCODER_BIDIRECTIONAL else ENCODER_HIDDEN_SIZE # Encoder를 양방향으로 한 경우 Decoder의 hidden 입력은 hidden_size * 2
EMBEDDIMG_DIM = 200

print(f'vocab_size: {VOCAB_SIZE}\n======================')

# Encoder 정의
encoder = Encoder(vocab_size=VOCAB_SIZE, 
                  hidden_size=ENCODER_HIDDEN_SIZE, 
                  embedding_dim=EMBEDDIMG_DIM, 
                  num_layers=1,
                  bidirectional=ENCODER_BIDIRECTIONAL)
# Decoder 정의
decoder = Decoder(vocab_size=VOCAB_SIZE, 
                  hidden_size=DECODER_HIDDEN_SIZE, 
                  embedding_dim=EMBEDDIMG_DIM, 
                  num_layers=1)

# Seq2Seq 생성
# encoder, decoder를 device 모두 지정
model = Seq2Seq(encoder, decoder, device)
print(model)

vocab_size: 15579
Seq2Seq(
  (encoder): Encoder(
    (embedding): Embedding(15579, 200, padding_idx=0)
    (gru): GRU(200, 512, bidirectional=True)
  )
  (decoder): Decoder(
    (embedding): Embedding(15579, 200, padding_idx=0)
    (dropout): Dropout(p=0, inplace=False)
    (gru): GRU(200, 1024)
    (classifier): Linear(in_features=1024, out_features=15579, bias=True)
  )
)


## loss함수, optimizer

In [69]:
lr = 0.001
optimizer = optim.Adam(model.parameters(), lr=lr)
loss_fn = nn.CrossEntropyLoss()

## train/evaluation 함수 정의

### train 함수정의

In [75]:
a, b = next(iter(train_loader))
b.shape

torch.Size([64, 25])

In [76]:
25 * 64

1600

In [ ]:
def train_fn(model, data_loader, optimizer, loss_fn, device):
    model.train()
    loss_list = []
    
    for X, y in data_loader:
        X, y = X.to(device), y.to(device)

        pred = model(X, y, teacher_forcing_ratio=0.5)
        # pred shape: (batch_size, seq_length, vocab_size:단어별확률이므로)  
        #            ex) (64, 25, 어휘수)
        
        yhat = pred.reshape(-1, pred.size(2))  # (batch * seq_len, 어휘수), (64*25, 15000)

        # 정답 shape변경
        # (batch_size*sequence_length) 로 변경 (64, 25) =>                  (64*25, )
        y = y.reshape(-1)           
        
        # Loss 계산
        loss = loss_fn(yhat, y)
        # gradient 계산
        loss.backward()
        #파라미터 업데이트
        optimizer.step()
        # gradient 초기화
        optimizer.zero_grad()
        
        loss_list.append(loss.item())
        
    return np.mean(loss_list)

### Test 함수

In [77]:
def test_fn(model, data_loader, loss_fn, device):
    model.eval()
    
    loss_list = []
    
    with torch.no_grad():
        for X, y in data_loader:
            X, y = X.to(device), y.to(device)
            pred = model(X, y)
            
            yhat = pred.reshape(-1, pred.shape[2])
            y = y.reshape(-1)
            
            # Loss 계산
            loss = loss_fn(yhat, y)
            
            loss_list.append(loss.item())
            
    return np.mean(loss_list)

### Training

In [ ]:
##########################################
# 학습은 GPU에서 해야 합니다. colab을 이용하세요
##########################################
import time

EPOCHS = 10
MODE_PATH = 'models/seq2seq-chatbot-model.pt'

best_loss = np.inf
s = time.time()
for epoch in range(EPOCHS):
    loss = train_fn(model, train_loader, optimizer, loss_fn, device)
    
    val_loss = test_fn(model, test_loader, loss_fn, device)
    
    if val_loss < best_loss:
        best_loss = val_loss
        torch.save(model, MODE_PATH)
        print(f"[{epoch+1} epoch 에서 저장]")
    
    if epoch % 5 == 0 or epoch == EPOCHS-1:
        print(f'epoch: {epoch+1}, loss: {loss:.4f}, val_loss: {val_loss:.4f}')

e = time.time()
print(e-s, "초")

# 결과확인

In [70]:
from torch.utils.data import SubsetRandomSampler 


def handle_special_tokens(decoded_string):
    """
    Subword 처리
    subword는 단어의 시작으로 쓰인 것과 중간 부분에 사용된 두가지 subword가 있다. 중간에 쓰인 subword는 `#`과 같은 특수문자로 시작 한다.
    tokenizer.decode() 결과 문자열은 subword의 특수문자('#')을 처리하지 않는다. 이것을 처리하는 함수
    ex) "이 기회 #는 내 #꺼 #야" ==> "이 기회는 내꺼야"
    
    Parameter
        decoded_string: str - Tokenizer가 decode한 중간 subword의 특수문자 처리가 안된 문자열. 
    Return
        str: subword 특수문자 처리한 문자열
    """
    tokens = decoded_string.split()
    
    new_tokens = []
    for token in tokens:
        if token.startswith('#'):
            if new_tokens: # 리스트에 토큰이 이미 있으면 마지막 토큰 뒤에 붙인다.  new_token[-1]을 조회해 그 뒤에 합친다. (#은 지우고)
                new_tokens[-1] += token[1:]
            else: # 첫번째 토큰일 경우는 그냥 #을 지우고 append한다. (첫 토큰으로 subword가 나올 수있기 때문에.)
                new_tokens.append(token[1:])
        else:
            new_tokens.append(token)
    return ' '.join(new_tokens)

# dataset에서 일부 데이터들을 가지고 확인
def random_evaluation(model, dataset, device, n=10):
    """
    Dataset에서 일부 질문-답변 쌍들을 가져다 모델에 질문을 넣어 추론한 결과와 함께 확인.
    Parameter
        model: 학습된 seq2seq 모델
        dataset: 질문-답변 쌍울 추출할 dataset
        device
        n: int - 추출할 질문-답변 쌍 개수 default: 10
    """
    n_samples = len(dataset)         # 총 데이터수
    indices = list(range(n_samples)) # index list
    np.random.shuffle(indices)      # Shuffle
    sampled_indices = indices[:n]   # n개 index
    
    # 샘플링한 데이터를 기반으로 DataLoader 생성
    ### DataLoader의 Sampler -> batch size만큼 Dataset에서 Data를 조회하는 방법을 정의한 객체.
    sampler = SubsetRandomSampler(sampled_indices) # index를 주면 그 index의 데이터들만 조회.
    sampled_dataloader = DataLoader(dataset, batch_size=10, sampler=sampler) 
    # sampler를 제공하면 그 sampler를 이용해 Dataset으로 부터 batch 개수 만큼 가져와 제공.
    
    model.eval()
    with torch.no_grad():
        for X, y in sampled_dataloader:
            X, y = X.to(device), y.to(device)        
            output = model(X, y, teacher_forcing_ratio=0) #추론 -> teacher forcing: 0
            # output: (number of samples, sequence_length, vocab_size)
            
            preds = output.cpu().numpy()  # to("cpu") == cpu()  (batch, seq, 총단어수)
            X = X.cpu().numpy()
            y = y.cpu().numpy()
            
            for i in range(n):
                print(f'질문   : {handle_special_tokens(tokenizer.decode(X[i]))}')
                print(f'답변   : {handle_special_tokens(tokenizer.decode(y[i]))}')
                print(f'예측답변: {handle_special_tokens(tokenizer.decode(preds[i].argmax(1)))}')  
                # pred[i] 한개 문장. (seq, 단어수) 단어수 기준 max index조회하면 각 seq의 token_id 배열이 나온다.
                print('==='*10)



In [83]:
tokenizer.decode(a[0].cpu().numpy())

'헤어진지 한달 .'

In [ ]:
# 테스트
MODE_PATH = 'models/seq2seq-chatbot-model.pt'
load_model = torch.load(MODE_PATH, map_location=device)
random_evaluation(load_model, test_set, device)

'헤어진지 한달 .'